In [ ]:
# General libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display
from sklearn.model_selection import train_test_split
import cv2

# CNN libraries
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import BatchNormalization, Conv2D, Dropout, MaxPooling2D, Dense, Flatten
from tensorflow.keras.models import Sequential 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence


# QOL libraries
import seaborn as sns
import tqdm
import random
import os

random.seed(13)

<h1>CNN Cancer Detection</h1>
  
  
#### Table of Contents

- [Problem and Data Description](#a)
- [EDA](#b)
- [DModel Architecture](#c)
- [Results and Analysis](#d)
    1. [Adding dropout layers](#da)
    2. [Adding more hidden layers](#db)
    3. [Increasing the filter size](#dc)
    4. [Increasing the learning rate](#dd)
    5. [Decreasing the learning rate](#de)
    6. [Introducing batch normalization](#df)
    7. [Changing the output activation function](#dg)
    
    
    
- [Conclusion](#e)

<h2>Problem and Data Description</h2><a name="a"></a>

This notebook is a submission for the Kaggle competition <a href = "https://www.kaggle.com/c/histopathologic-cancer-detection/overview">Histopathologic Cancer Detection</a>.  The premise of this competition is to create a convolutional neural network that will take a series of images taken from pathology scans, and output a prediction of whether or not the sample in question contains signs of metastatic cancer.

The data consists of two parts, and the first is a large folder of histopathologic scans of lymph nodes.  Each image has their respective image ID as their name, and is saved in a .tif format.  The images are also perfectly squared with dimensions of 96 pixels in width, 96 pixels in height, and has a depth of 3.  The folder is further split into a set of images for training our models, as well as another set for testing used to create the competition submission.

The second part of the data is a labels dataset that lists the ids of each image in the training image set and their respective labels on whether or not there is evidence of cancer tissues in the center 32 x 32 pixel region of the iamge.  A label of 0 states that there is no sign of cancer in the respective scan, while a label of 1 states that there is signs of cancer.

In total, our training data has 220,025 images to use, and as we are not professional doctors, to the majority of us these images would make no sense and thus we will rely on the model to learn to predict our response.

<h2>Exploratory Data Analysis</h2><a name="b"></a>

As in this situation we are working with images as our source of data, our main dataframe will be built from the dataset of labels which we will use to keep track of the image IDs, their status, as well as their purpose for training and validation.

For the purposes of this submission, we will be cutting down the number of images used to train from 220,025 to 13,000 in order to cut down on model training time to allow for more chances of testing a wider variety of hyper parameters.

In [ ]:
path = 'data/histopathologic-cancer-detection/'
df = pd.read_csv(path + 'train_labels.csv', dtype = str)
df['file'] = df['id'] + '.tif'

#
df = df.sample(n = 13000, random_state = 13)

In [ ]:
def match_image(pic_id, train_test_validation = 'train'):
    return path + train_test_validation + '/' + pic_id + '.tif'

img = Image.open(match_image('f38a6374c348f90b587e046aac6079959adf3835'), 'r')
plt.imshow(img)

Here are example images of when there are no sign of metastatic cancer.

In [ ]:
fig, axs = plt.subplots(2, 3, figsize = (10, 6))

for i, j in zip(axs.ravel(), df[df['label'] == '0'].sample(6, random_state = 17)['id']):
    im = Image.open(match_image(j), 'r')
    i.imshow(im)

plt.show()

Here are example images of when there are signs of metastatic cancer.

In [ ]:
fig, axs = plt.subplots(2, 3, figsize = (10, 6))

for i, j in zip(axs.ravel(), df[df['label'] == '1'].sample(6, random_state = 17)['id']):
    img = Image.open(match_image(j), 'r')
    i.imshow(img)

plt.show()

Now we check the distribution of our labeled classes, as well as some other standard checks such as ensuring that there are no Null or duplicate values.

In [ ]:
df['label'].value_counts(normalize = True).plot(kind = 'bar')

In [ ]:
df.isnull().sum()

In [ ]:
df[df.duplicated()]

From the barchart, we can see that there is a small imbalance between our class disparity which can be adjusted for later on.  Fortunately, the data does not come with any missing or duplicate values.

<h2>DModel Architecture</h2><a name="c"></a>

To start, the 'train' data will be further split into a training dataset and a validation dataset, with a 8:2 split.   Furthermore, the model will be trained on the training set and scored on the validation dataset.

In [ ]:
train, val = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

For our convolutional neural network we will be using a sequential model with the simple structure of convolution-convolution-MaxPool.  We will create a basic, barebones model with default hyperparameters as well as only one hidden layer.  This is to test how the model to interact with images.

In [ ]:
filter_1 = 32
filter_size = ( 3, 3)
img_width = img.width # 96
img_height = img.height # 96
img_depth = 3 # RGB
pool_size = ( 2, 2)

model = Sequential()

model.add(Conv2D(filter_1, filter_size, activation = 'relu', input_shape = (img_width, img_height, img_depth)))
model.add(Conv2D(filter_1, filter_size, activation = 'relu'))
model.add(MaxPooling2D(pool_size)) 

model.add(Flatten())
model.add(Dense(filter_1 * 2, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(Adam(learning_rate = 0.001), loss='binary_crossentropy', metrics=['accuracy'])

As I struggled with figuring out how to input images into my Sequential CNN model, I resorted to referencing the code in the notebook <a href= "https://www.kaggle.com/code/fadhli/starter-code-keras-resnet50-0-9275-lb/notebook">Starter Code Keras</a> to view how the ImageDataGenerator function was utilized.

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=lambda x:(x - x.mean()) / x.std() if x.std() > 0 else x,
                                   horizontal_flip=True,
                                   vertical_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=lambda x:(x - x.mean()) / x.std() if x.std() > 0 else x)

train_generator = train_datagen.flow_from_dataframe(
    dataframe = train,
    directory= path + 'train/',
    x_col='file',
    y_col='label',
    has_ext=False,
    batch_size=32,
    seed=2018,
    shuffle=True,
    class_mode='binary',
    target_size=(96,96),
    validate_filenames = False
)

valid_generator = test_datagen.flow_from_dataframe(
    dataframe = val,
    directory= path + 'train/',
    x_col='file',
    y_col='label',
    has_ext=False,
    batch_size=32,
    seed=2018,
    shuffle=False,
    class_mode='binary',
    target_size=(96,96),
    validate_filenames = False
)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1, restore_best_weights=True)
reducel = ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.1)

In [ ]:
history = model.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN, 
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5,
                   callbacks=[reducel, earlystopper])

In [ ]:
plt.plot(history.history['val_accuracy'], label = 'validation accuracy')
plt.plot(history.history['accuracy'], label = 'training accuracy')
plt.legend()
plt.show()

From our accuracy diagram, we can tell that with our base model it can reach a relatively high training accuracy of over 0.84.  However, it is noticeably higher than than our validation accuracy which is a sign of a small bit of overfitting.  As this is just a base model, we can slowly start tweaking hyperparameters and the architecture.

<h2>Results and Analysis</h2><a name="d"></a>

With the results from our base model, we will now undergo hyperparameter tuning in an attempt to improve our validation accuracy.  The ideas that we will test are:

1. Adding dropout layers
2. Adding more hidden layers
3. Increasing the filter size
4. Increasing the learning rate
5. Decreasing the learning rate
6. Introducing batch normalization
7. Changing the output activation function

Like the previous model, we will be mainly focusing on the accuracy statistic when the model is tested on the validation data set.

<h3>1.  Adding dropout layers.</h3><a name="da"></a>

The first step will be to add dropout layers after hidden layer and output layer.  This is to help with regularization purposes.  Thus we can see while the overall model performance does not change much, the difference between the training and validation scores have been minimalized by a significant amount.

In [ ]:
filter_1 = 32
filter_size = ( 3, 3)
img_width = img.width # 96
img_height = img.height # 96
img_depth = 3 # RGB
dropout_conv = 0.3
dropout_dense = 0.5
pool_size = ( 2, 2)

model_1 = Sequential()

model_1.add(Conv2D(filter_1, filter_size, activation = 'relu', input_shape = (img_width, img_height, img_depth)))
model_1.add(Conv2D(filter_1, filter_size, activation = 'relu'))
model_1.add(MaxPooling2D(pool_size)) 
model_1.add(Dropout(dropout_conv))

model_1.add(Flatten())
model_1.add(Dense(filter_1 * 2, activation = 'relu'))
model_1.add(Dropout(dropout_dense))
model_1.add(Dense(1, activation = 'sigmoid'))

model_1.compile(Adam(learning_rate = 0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history_1 = model_1.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN, 
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5,
                   callbacks=[reducel, earlystopper])

In [ ]:
plt.plot(history_1.history['val_accuracy'], label = 'validation accuracy')
plt.plot(history_1.history['accuracy'], label = 'training accuracy')
plt.legend()
plt.show()

<h3>2. Adding more hidden layers</h3><a name="db"></a>

As our base model only has one hidden layer, we can significantly increase the training prowress by adding more hidden layers so that the model can better learn from the data.  We add two more layers, each with a filter size that is twice the size of the previous layer.  From the results, while the computation time increased, there is also an increase in both the training and validation accuracy scores.

In [ ]:
filter_1 = 32
filter_2 = 64
filter_3 = 128
filter_size = ( 3, 3)
img_width = img.width # 96
img_height = img.height # 96
img_depth = 3 # RGB
dropout_conv = 0.3
dropout_dense = 0.5
pool_size = ( 2, 2)

model_2 = Sequential()

model_2.add(Conv2D(filter_1, filter_size, activation = 'relu', input_shape = (img_width, img_height, img_depth)))
model_2.add(Conv2D(filter_1, filter_size, activation = 'relu'))
model_2.add(MaxPooling2D(pool_size)) 
model_2.add(Dropout(dropout_conv))

model_2.add(Conv2D(filter_2, filter_size, activation = 'relu'))
model_2.add(Conv2D(filter_2, filter_size, activation = 'relu'))
model_2.add(MaxPooling2D(pool_size)) 
model_2.add(Dropout(dropout_conv))

model_2.add(Conv2D(filter_3, filter_size, activation = 'relu'))
model_2.add(Conv2D(filter_3, filter_size, activation = 'relu'))
model_2.add(MaxPooling2D(pool_size)) 
model_2.add(Dropout(dropout_conv))

model_2.add(Flatten())
model_2.add(Dense(256, activation = 'relu'))
model_2.add(Dropout(dropout_dense))
model_2.add(Dense(1, activation = 'sigmoid'))

model_2.compile(Adam(learning_rate = 0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history_2 = model_2.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN, 
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5,
                   callbacks=[reducel, earlystopper])

In [ ]:
plt.plot(history_2.history['val_accuracy'], label = 'validation accuracy')
plt.plot(history_2.history['accuracy'], label = 'training accuracy')
plt.legend()
plt.show()

<h3>3. Increasing the filter size</h3><a name="dc"></a>

Next we change the size of our filters from 3x3 to a 7x7 grid.  However, our results did not improve and thus this change will be ignored.

In [ ]:
filter_1 = 32
filter_2 = 64
filter_3 = 128
filter_size = ( 7, 7)
img_width = img.width # 96
img_height = img.height # 96
img_depth = 3 # RGB
dropout_conv = 0.3
dropout_dense = 0.5
pool_size = ( 2, 2)

model_3 = Sequential()

model_3.add(Conv2D(filter_1, filter_size, activation = 'relu', input_shape = (img_width, img_height, img_depth)))
model_3.add(Conv2D(filter_1, filter_size, activation = 'relu'))
model_3.add(MaxPooling2D(pool_size)) 
model_3.add(Dropout(dropout_conv))

model_3.add(Conv2D(filter_2, filter_size, activation = 'relu'))
model_3.add(Conv2D(filter_2, filter_size, activation = 'relu'))
model_3.add(MaxPooling2D(pool_size)) 
model_3.add(Dropout(dropout_conv))

model_3.add(Conv2D(filter_3, filter_size, activation = 'relu'))
model_3.add(Conv2D(filter_3, filter_size, activation = 'relu'))
model_3.add(MaxPooling2D(pool_size)) 
model_3.add(Dropout(dropout_conv))

model_3.add(Flatten())
model_3.add(Dense(256, activation = 'relu'))
model_3.add(Dropout(dropout_dense))
model_3.add(Dense(1, activation = 'sigmoid'))

model_3.compile(Adam(learning_rate = 0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history_3 = model_3.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN, 
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5,
                   callbacks=[reducel, earlystopper])

In [ ]:
plt.plot(history_3.history['val_accuracy'], label = 'validation accuracy')
plt.plot(history_3.history['accuracy'], label = 'training accuracy')
plt.legend()
plt.show()

<h3>4. Increasing the learning rate</h3><a name="dd"></a>

We attempted to test tweaking the learning rate by increasing it from 0.001 to 0.01.  However the results turned out to be disastrous with a very stable low score.  

In [ ]:
filter_1 = 32
filter_2 = 64
filter_3 = 128
filter_size = ( 3, 3)
img_width = img.width # 96
img_height = img.height # 96
img_depth = 3 # RGB
dropout_conv = 0.3
dropout_dense = 0.5
pool_size = ( 2, 2)

model_4 = Sequential()

model_4.add(Conv2D(filter_1, filter_size, activation = 'relu', input_shape = (img_width, img_height, img_depth)))
model_4.add(Conv2D(filter_1, filter_size, activation = 'relu'))
model_4.add(MaxPooling2D(pool_size)) 
model_4.add(Dropout(dropout_conv))

model_4.add(Conv2D(filter_2, filter_size, activation = 'relu'))
model_4.add(Conv2D(filter_2, filter_size, activation = 'relu'))
model_4.add(MaxPooling2D(pool_size)) 
model_4.add(Dropout(dropout_conv))

model_4.add(Conv2D(filter_3, filter_size, activation = 'relu'))
model_4.add(Conv2D(filter_3, filter_size, activation = 'relu'))
model_4.add(MaxPooling2D(pool_size)) 
model_4.add(Dropout(dropout_conv))

model_4.add(Flatten())
model_4.add(Dense(256, activation = 'relu'))
model_4.add(Dropout(dropout_dense))
model_4.add(Dense(1, activation = 'sigmoid'))

model_4.compile(Adam(learning_rate = 0.01), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history_4 = model_4.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN, 
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5,
                   callbacks=[reducel, earlystopper])

In [ ]:
plt.plot(history_4.history['val_accuracy'], label = 'validation accuracy')
plt.plot(history_4.history['accuracy'], label = 'training accuracy')
plt.legend()
plt.show()

<h3>5. Decreasing the learning rate</h3><a name="de"></a>

Instead of increasing the learning rate, we test what happens when the learning rate is decreased to 0.0001.  While not as bad as when the learning rate is increased, there are signs of severe overfitting.  Thus in the end, our default learning rate of 0.001 is a good rate to settle with.

In [ ]:
filter_1 = 32
filter_2 = 64
filter_3 = 128
filter_size = ( 3, 3)
img_width = img.width # 96
img_height = img.height # 96
img_depth = 3 # RGB
dropout_conv = 0.3
dropout_dense = 0.5
pool_size = ( 2, 2)

model_5 = Sequential()

model_5.add(Conv2D(filter_1, filter_size, activation = 'relu', input_shape = (img_width, img_height, img_depth)))
model_5.add(Conv2D(filter_1, filter_size, activation = 'relu'))
model_5.add(MaxPooling2D(pool_size)) 
model_5.add(Dropout(dropout_conv))

model_5.add(Conv2D(filter_2, filter_size, activation = 'relu'))
model_5.add(Conv2D(filter_2, filter_size, activation = 'relu'))
model_5.add(MaxPooling2D(pool_size)) 
model_5.add(Dropout(dropout_conv))

model_5.add(Conv2D(filter_3, filter_size, activation = 'relu'))
model_5.add(Conv2D(filter_3, filter_size, activation = 'relu'))
model_5.add(MaxPooling2D(pool_size)) 
model_5.add(Dropout(dropout_conv))

model_5.add(Flatten())
model_5.add(Dense(256, activation = 'relu'))
model_5.add(Dropout(dropout_dense))
model_5.add(Dense(1, activation = 'sigmoid'))

model_5.compile(Adam(learning_rate = 0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history_5 = model_5.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN, 
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5,
                   callbacks=[reducel, earlystopper])

In [ ]:
plt.plot(history_5.history['val_accuracy'], label = 'validation accuracy')
plt.plot(history_5.history['accuracy'], label = 'training accuracy')
plt.legend()
plt.show()

<h3>6. Introducing batch normalization</h3><a name="df"></a>

Similar to dropout layers, batch normalization is another regularization method we can employ.  From the results, both the training accuracy and the validation accuracy have increased slightly over the previous methods.  Thus this is a change we can keep.

In [ ]:
filter_1 = 32
filter_2 = 64
filter_3 = 128
filter_size = ( 3, 3)
img_width = img.width # 96
img_height = img.height # 96
img_depth = 3 # RGB
dropout_conv = 0.3
dropout_dense = 0.5
pool_size = ( 2, 2)

model_6 = Sequential()

model_6.add(Conv2D(filter_1, filter_size, activation = 'relu', input_shape = (img_width, img_height, img_depth)))
model_6.add(BatchNormalization())
model_6.add(Conv2D(filter_1, filter_size, activation = 'relu'))
model_6.add(BatchNormalization())
model_6.add(MaxPooling2D(pool_size)) 
model_6.add(Dropout(dropout_conv))

model_6.add(Conv2D(filter_2, filter_size, activation = 'relu'))
model_6.add(BatchNormalization())
model_6.add(Conv2D(filter_2, filter_size, activation = 'relu'))
model_6.add(BatchNormalization())
model_6.add(MaxPooling2D(pool_size)) 
model_6.add(Dropout(dropout_conv))

model_6.add(Conv2D(filter_3, filter_size, activation = 'relu'))
model_6.add(BatchNormalization())
model_6.add(Conv2D(filter_3, filter_size, activation = 'relu'))
model_6.add(BatchNormalization())
model_6.add(MaxPooling2D(pool_size)) 
model_6.add(Dropout(dropout_conv))

model_6.add(Flatten())
model_6.add(Dense(256, activation = 'relu'))
model_6.add(BatchNormalization())
model_6.add(Dropout(dropout_dense))
model_6.add(Dense(1, activation = 'sigmoid'))

model_6.compile(Adam(learning_rate = 0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history_6 = model_6.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN, 
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5,
                   callbacks=[reducel, earlystopper])

In [ ]:
plt.plot(history_6.history['val_accuracy'], label = 'validation accuracy')
plt.plot(history_6.history['accuracy'], label = 'training accuracy')
plt.legend()
plt.show()

<h3>7. Changing the output activation function</h3><a name="dg"></a>

For our final hyperparameter tuning, we will try changing the activation function of our output layer from a sigmoid activation to a softmax activation.  However as seen from our results, a model with a softmax activation function proved to be the worst out of all our current models.  This is most likely due to sigmoid being preferable for binary outputs while softmax is more geared for numerous categories.

In [ ]:
filter_1 = 32
filter_2 = 64
filter_3 = 128
filter_size = ( 3, 3)
img_width = img.width # 96
img_height = img.height # 96
img_depth = 3 # RGB
dropout_conv = 0.3
dropout_dense = 0.5
pool_size = ( 2, 2)

model_7 = Sequential()

model_7.add(Conv2D(filter_1, filter_size, activation = 'relu', input_shape = (img_width, img_height, img_depth)))
model_7.add(BatchNormalization())
model_7.add(Conv2D(filter_1, filter_size, activation = 'relu'))
model_7.add(BatchNormalization())
model_7.add(MaxPooling2D(pool_size)) 
model_7.add(Dropout(dropout_conv))

model_7.add(Conv2D(filter_2, filter_size, activation = 'relu'))
model_7.add(BatchNormalization())
model_7.add(Conv2D(filter_2, filter_size, activation = 'relu'))
model_7.add(BatchNormalization())
model_7.add(MaxPooling2D(pool_size)) 
model_7.add(Dropout(dropout_conv))

model_7.add(Conv2D(filter_3, filter_size, activation = 'relu'))
model_7.add(BatchNormalization())
model_7.add(Conv2D(filter_3, filter_size, activation = 'relu'))
model_7.add(BatchNormalization())
model_7.add(MaxPooling2D(pool_size)) 
model_7.add(Dropout(dropout_conv))

model_7.add(Flatten())
model_7.add(Dense(256, activation = 'relu'))
model_7.add(BatchNormalization())
model_7.add(Dropout(dropout_dense))
model_7.add(Dense(1, activation = 'softmax'))

model_7.compile(Adam(learning_rate = 0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history_7 = model_7.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN, 
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5,
                   callbacks=[reducel, earlystopper])

In [ ]:
plt.plot(history_7.history['val_accuracy'], label = 'validation accuracy')
plt.plot(history_7.history['accuracy'], label = 'training accuracy')
plt.legend()
plt.show()

<h2>Conclusion</h2><a name="e"></a>

In the end, the 6th model proved to have the best performance.  This was the model that had dropout layers, 3 hidden layers, 3x3 filter size, a learning rate of 0.001, batch normalization, and a sigmoid activation function.

Some key takeaways from this was the importance of regularization methods, as without them to help combat overfitting there would be a large difference in performance between using the training data versus any other data.  Another thing of note is the learning rate, as a higher as well as lower learning rate lead to a significant decrease in performance.  However as this is a spectrum rather than discrete category, some more testing could be done to find if there is a more optimal value than 0.001.

It is also of note that for the sake of ease of access, we only used 13,000 of the original 220,025 images to train our model with.  If more data was utilized, some hyperparameters could have either a more positive or negative effect on model performance, which is an improvement that could be tested in the future.

In [ ]:
hst = [ history_1, history_2, history_3, history_4, history_5, history_6, history_7]
hst_val = [ max(i.history['val_accuracy']) for i in hst ]

plt.scatter(range(1,8), hst_val)
plt.xlabel('model #')
plt.ylabel('max')
plt.show()

In [ ]:
model_f = model_6
history_f = history_6

test_generator = test_datagen.flow_from_directory(
    directory= path + 'test1/',
    batch_size=1,
    seed=2018,
    shuffle=False,
    class_mode='binary',
    target_size=(96,96)
)

With our final model chosen model, we can create our predictions on the proper test images and save them as our submission 'output.csv'.

In [ ]:
p = model_f.predict(test_generator, steps = 57458, verbose = 1)

ps = [i for j in p for i in j]
preds = pd.DataFrame({'id': test_generator.filenames, 'label':ps})
preds['id'] = preds['id'].str.slice(5, -4)

preds.to_csv('output.csv', index = False)

From our submission, the model ends up with a score of 0.8636 in the leaderboards.